# Capstone Project: Interpretation and Solution

### Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mpimg
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg') # Thanks Riley!
from functools import partial
import pyproj
from shapely.ops import transform
from shapely.geometry import Point
import geopy.distance

<ipython-input-1-44ed263ceead>:7: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg') # Thanks Riley!


### Datasets

In [2]:
capstone_df = pd.read_csv("../data/final_df_min.csv")

In [3]:
lat_long_venue = pd.read_csv("../data/location_with_google_ratings.csv")

In [4]:
distance_df = pd.read_csv("../data/distance_df.csv")

In [5]:
capstone_df["target"] = capstone_df["categories"].map(lambda x: 1 if x == "Coffee Shop" else 0)

In [6]:
capstone_df = pd.merge(capstone_df,
                       lat_long_venue[["area","categories","name","lat_venue","long_venue"]],
                       on = ["area","categories","name"],
                       how = "left"
                      )

In [7]:
distance_df = pd.merge(distance_df,
                       lat_long_venue[["area","categories","name","lat_venue","long_venue"]],
                       on = ["area","name","lat_venue","long_venue"],
                       how = "left"
                      )

In [8]:
capstone_df.head()

,area,categories,name,ATM,Acai House,Accessories Store,Acupuncturist,Afghan Restaurant,African Restaurant,Airport,...,user_rate_as_pct_pop,freq,venues_per_capita,cs_per_capita,price_level,rating,user_ratings_total,target,lat_venue,long_venue
0,Alameda,Afghan Restaurant,Q's Halal Chicken,19511.372574,13979.911196,11198.426865,62759.515967,0.000000,13365.306366,15743.376839,...,0.007021,0.050000,0.001288,0.000052,1.0,4.4,545.0,0,37.764583,-122.243813
1,Alameda,Airport Service,JSX (JetsuiteX),14136.570906,18560.157257,16007.462666,57497.347395,5382.747523,16368.791699,10389.712669,...,0.000000,0.111111,0.001288,0.000052,0.0,0.0,0.0,0,37.726916,-122.205344
2,Alameda,American Restaurant,Scolari's Good Eats,19382.759806,13993.863443,11400.194018,62593.193753,211.060273,13278.261825,15621.447513,...,0.006995,0.005222,0.001288,0.000052,2.0,4.6,543.0,0,37.762714,-122.244250
3,Alameda,American Restaurant,Scolari's Good Eats,19382.759806,13993.863443,11400.194018,62593.193753,211.060273,13278.261825,15621.447513,...,0.006995,0.005222,0.001288,0.000052,2.0,4.6,543.0,0,37.762714,-122.244250
4,Alameda,American Restaurant,Scolari's Good Eats,19382.759806,13993.863443,11400.194018,62593.193753,211.060273,13278.261825,15621.447513,...,0.006995,0.005222,0.001288,0.000052,2.0,4.6,543.0,0,37.762714,-122.244250


### Final Model Interpretation

In [9]:
filters = ["Park","Pizza Place","Ice Cream Shop","Mexican Restaurant","Bakery"]

In [10]:
areas_df = capstone_df[capstone_df["categories"].isin(filters)][["name","area","lat_venue","long_venue","Coffee Shop"]]

In [11]:
areas_df.reset_index(inplace = True, drop = True)

In [12]:
areas_df["geopoint"] = [(areas_df.loc[i,"lat_venue"], areas_df.loc[i,"long_venue"]) for i in areas_df.index]

In [13]:
areas_df["va"] = areas_df["name"] + ", " + areas_df["area"]

In [14]:
areas_df.set_index("va", inplace = True)

In [15]:
areas_df.drop_duplicates(inplace = True)

In [16]:
len(areas_df.index)

1538

In [17]:
venues = list(areas_df.index.unique())

In [18]:
coffee_venues = distance_df[(distance_df["categories"] == "Coffee Shop")]

In [19]:
areas_df.head()

,name,area,lat_venue,long_venue,Coffee Shop,geopoint
va,,,,,,
"Feel Good Bakery, Alameda",Feel Good Bakery,Alameda,37.767751,-122.239502,1301.741806,"(37.76775069938167, -122.23950204783034)"
"Feel Good Kitchen, Alameda",Feel Good Kitchen,Alameda,37.756228,-122.233648,2159.632421,"(37.75622819247969, -122.23364814597196)"
"Cookiebar Creamery, Alameda",Cookiebar Creamery,Alameda,37.771530,-122.277330,1531.854302,"(37.77153, -122.27733)"
"Tucker's Ice Cream, Alameda",Tucker's Ice Cream,Alameda,37.763843,-122.243297,1021.106654,"(37.76384342959636, -122.24329684639896)"
"Chipotle Mexican Grill, Alameda",Chipotle Mexican Grill,Alameda,37.756121,-122.250114,1248.348611,"(37.7561207, -122.2501137)"


In [20]:
remove_ls = []

for venue in venues.copy():
    for dist in coffee_venues[venue]:
        if dist <= areas_df.loc[venue,"Coffee Shop"]:
            remove_ls.append(venue)

In [21]:
ven_ls = [venue for venue in venues if venue not in remove_ls]

In [22]:
len(ven_ls)

265

In [23]:
fin_areas = areas_df[areas_df.index.isin(ven_ls)]   

In [24]:
# Source: https://gis.stackexchange.com/questions/289044/creating-buffer-circle-x-kilometers-from-point-using-python

proj_wgs84 = pyproj.Proj('+proj=longlat +datum=WGS84')


def geodesic_point_buffer(lat, lon, m):
    aeqd_proj = '+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0'
    project = partial(
        pyproj.transform,
        pyproj.Proj(aeqd_proj.format(lat=lat, lon=lon)),
        proj_wgs84)
    buf = Point(0, 0).buffer(m)
    return transform(project, buf).exterior.coords[:]

In [25]:
california_img=mpimg.imread('../graphics/bay_area_map.png')

fig = plt.figure(figsize=(10,7))
ax = fig.add_subplot()

radius_ls = []

for lat,long in fin_areas["geopoint"]:
    ls_radius = geodesic_point_buffer(lat, long, fin_areas["Coffee Shop"].mean())
    radius_ls.append(ls_radius)
    
long_rad = []
lat_rad = []

for points in radius_ls:
    for long,lat in points:
        long_rad.append(long)
        lat_rad.append(lat)

ax.scatter(long_rad, 
        lat_rad, 
        c="black",
        s = 1,
        alpha = 1,
        label = "Radius")

ax.scatter(fin_areas["long_venue"], 
        fin_areas["lat_venue"], 
        alpha = 1,
        s = 8,
        label = "Related Venues")

ax.scatter(capstone_df[capstone_df["categories"] == "Coffee Shop"]["long_venue"], 
        capstone_df[capstone_df["categories"] == "Coffee Shop"]["lat_venue"], 
        c = "r",
        alpha = 1,
        s = 4,
        label = "Coffee Shop")

plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.imshow(california_img, extent=[-122.6445, -121.5871, 37.1897, 38.2033], alpha=0.5)
plt.title("Recommended Areas")
plt.ylabel("Latitude", fontsize=14)
plt.xlabel("Longitude", fontsize=14)

plt.show()